# HW 4

In [101]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import math

## Part A

In [102]:
def creategraphfromtsv_old(filename):
    df = pd.read_csv(filename, sep='\t', names = ['source', 'target'])
    graph = nx.convert_matrix.from_pandas_edgelist(df)
    return graph

In [103]:
G_old = creategraphfromtsv_old('old_edges.txt')

G_old

## Part B

In [118]:
def creategraphfromtsv_new(filename):
    df = pd.read_csv(filename, sep='\t', names = ['source', 'target'])
    graph = nx.convert_matrix.from_pandas_edgelist(df) 
    atleast10 = [node for node,degree in dict(graph.degree()).items() if degree >= 10]
    
    return graph, atleast10

In [119]:
G_new, atleast10 = creategraphfromtsv_new('new_edges.txt')

G_new

## Part C - Common Friends

In [120]:
def numcommonfriends(G, x1, x2):
    n1 = set(G.neighbors(x1))
    n2 = set(G.neighbors(x2))
    
    intersection = n2.intersection(n1)
    
    return len(intersection)

def common_friends_number(G, X):
    recs = {i:0 for i in G.nodes}
    del recs[X]
    for author in recs:
        if not G.has_edge(X, author): # if they are not already friends
            recs[author] = numcommonfriends(G, X, author)
        else:
            recs[author] = -1 # if they are friends, assign -1 so can remove it
    
    recs = {key:val for key, val in recs.items() if val != -1}
        
    sortedrecs = {k: v for k, v in sorted(recs.items(), key=lambda item: item[1], reverse=True)}
    sortedrecslist = list(sortedrecs.keys())
    return sortedrecslist, sortedrecs

recs, dicts = common_friends_number(G_old, 'Francis R. Bach')
recs[:15]

['Antoine Bordes',
 'Asma Rabaoui',
 'Emmanuel Duflos',
 'Eric P. Xing',
 'Michael I. Jordan',
 'François Laviolette',
 'Mario Marchand',
 'Sara Shanian',
 'Milan Vojnovic',
 'Mohammad Ghavamzadeh',
 'Martin Jaggi',
 'Chengtao Li',
 'David M. Mimno',
 'Perry R. Cook',
 'Zaïd Harchaoui']

## Part D - Jaccard's Index

In [121]:
def jaccard_index(G, X):
    recs = {i:0 for i in G.nodes}
    del recs[X]
    
    for author in recs:
        if not G.has_edge(X, author): # if they are not already friends
            listt = nx.jaccard_coefficient(G, [(X, author)])
            for u, v, i in listt:
                recs[author] = i
        else:
            recs[author] = -1 # if they are friends, assign -1 so can remove it
            
    recs = {key:val for key, val in recs.items() if val != -1}
        
    sortedrecs = {k: v for k, v in sorted(recs.items(), key=lambda item: item[1], reverse=True)}
    sortedrecslist = list(sortedrecs.keys())
    return sortedrecslist, sortedrecs

recs, dicts = jaccard_index(G_old, 'Francis R. Bach')
recs[:15]

['Asma Rabaoui',
 'Emmanuel Duflos',
 'Antoine Bordes',
 'Perry R. Cook',
 'Guillaume Bourque',
 'Sara Shanian',
 'Chengtao Li',
 'Stephan Mandt',
 'Patrick Pletscher',
 'Milan Vojnovic',
 'Samuel Gershman',
 'Justin Solomon',
 'Hyun Oh Song',
 'Zaïd Harchaoui',
 'Mario Marchand']

## Part E - Adamic/Adar Index

In [122]:
def adamic_adar_index(G, X):
    recs = {i:0 for i in G.nodes}
    del recs[X]
    
    for author in recs:
        if not G.has_edge(X, author): # if they are not already friends
            listt = nx.adamic_adar_index(G, [(X, author)])
            for u, v, i in listt:
                recs[author] = i
        else:
            recs[author] = -1 # if they are friends, assign -1 so can remove it
            
    recs = {key:val for key, val in recs.items() if val != -1}
        
    sortedrecs = {k: v for k, v in sorted(recs.items(), key=lambda item: item[1], reverse=True)}
    sortedrecslist = list(sortedrecs.keys())
    return sortedrecslist, sortedrecs

recs, dicts = adamic_adar_index(G_old, 'Francis R. Bach')
recs[:15]

['Antoine Bordes',
 'Asma Rabaoui',
 'Emmanuel Duflos',
 'Guillaume Bourque',
 'Milan Vojnovic',
 'François Laviolette',
 'Mario Marchand',
 'Sara Shanian',
 'Mohammad Ghavamzadeh',
 'Justin Solomon',
 'Martin Jaggi',
 'Patrick Pletscher',
 'Justin Domke',
 'Tibério S. Caetano',
 'Yann LeCun']

## Part F - Validation

### 10 Recommendation Method

In [123]:
def val_10recs(recfunction, oldgraph, newgraph):
    vals = []
    for author in atleast10:      
        recommendations, dicts = recfunction(oldgraph, author)
        top10_recommendations = set(recommendations[:10])
        newconnections = set(newgraph.neighbors(author)) 
        vals.append(len(top10_recommendations.intersection(newconnections)))
        
    avg = sum(vals) / len(vals)
        
    return avg

commonfriends_efficiency = val_10recs(common_friends_number, G_old, G_new)
jaccard_efficiency = val_10recs(jaccard_index, G_old, G_new)
adamic_efficiency = val_10recs(adamic_adar_index, G_old, G_new)

print('Common Friends 10 Recs Efficiency:', commonfriends_efficiency)
print('Jaccards Index 10 Recs Efficiency:', jaccard_efficiency)
print('Adamic/Adar 10 Recs Efficiency:', adamic_efficiency)

Common Friends 10 Recs Efficiency: 1.1951219512195121
Jaccards Index 10 Recs Efficiency: 0.8048780487804879
Adamic/Adar 10 Recs Efficiency: 1.024390243902439


### Rank Method

In [124]:
def val_rank(recfunction, oldgraph, newgraph):
    vals = []
    
    for author in atleast10:
        recommendations, dicts = recfunction(oldgraph, author)
        
        for newconnection in newgraph.neighbors(author):
            index = recommendations.index(newconnection)
            vals.append(index)
    
    return sum(vals) / len(vals)
        
commonfriends_efficiency = val_rank(common_friends_number, G_old, G_new)
jaccard_efficiency = val_rank(jaccard_index, G_old, G_new)
adamic_efficiency = val_rank(adamic_adar_index, G_old, G_new)

print('Common Friends Rank Efficiency:', commonfriends_efficiency)
print('Jaccards Index Rank Efficiency:', jaccard_efficiency)
print('Adamic/Adar Rank Efficiency:', adamic_efficiency)

Common Friends Rank Efficiency: 1690.9830220713072
Jaccards Index Rank Efficiency: 1696.93039049236
Adamic/Adar Rank Efficiency: 1694.188455008489


## Part E - Bonus

I felt as though the Adamic/Adar Index was a good metric for likely predicting correct recommendations. However, I figured why does the algorithm need to take the logarithm of the intersection of the number of neighbors? I felt as though simply summing up 1/neighbors(Z) would make more sense. So, I did some research to see if this metric exists. I eventually found that it does exist, and it is called the "resource allocation index". Thus, I impliment it below using the nx built in method to calculate it. Below that are its evaluation metrics.

In [128]:
def bonus(G, X):
    recs = {i:0 for i in G.nodes}
    del recs[X]
    
    for author in recs:
        if not G.has_edge(X, author): # if they are not already friends
            listt = nx.resource_allocation_index(G, [(X, author)])
            for u, v, i in listt:
                recs[author] = i
        else:
            recs[author] = -1 # if they are friends, assign -1 so can remove it
        
    recs = {key:val for key, val in recs.items() if val != -1}
    
    sortedrecs = {k: v for k, v in sorted(recs.items(), key=lambda item: item[1], reverse=True)}
    sortedrecslist = list(sortedrecs.keys())
    return sortedrecslist, sortedrecs

recs, dicts = bonus(G_old, 'Francis R. Bach')
recs[:15]

['Guillaume Bourque',
 'Milan Vojnovic',
 'Antoine Bordes',
 'Asma Rabaoui',
 'Emmanuel Duflos',
 'François Laviolette',
 'Mario Marchand',
 'Sara Shanian',
 'Mohammad Ghavamzadeh',
 'Justin Domke',
 'Tibério S. Caetano',
 'Yann LeCun',
 'Y-Lan Boureau',
 'Justin Solomon',
 'Andrew W. Fitzgibbon']

In [129]:
tenrecsefficency = val_10recs(bonus, G_old, G_new)
print('Val 10 recs Efficiency:', tenrecsefficency)

ranksefficiency = val_rank(bonus, G_old, G_new)
print('Rank effiency:', ranksefficiency)

Val 10 recs Efficiency: 0.7804878048780488
Rank effiency: 1695.030560271647


# Trash Below This

In [78]:
def calculateindex(G, x1, x2):    
    n1 = set(G.neighbors(x1))
    n2 = set(G.neighbors(x2))
    
    union = n1.union(n2)
    intersection = n1.intersection(n2)
    
    return len(intersection) / len(union)

def jaccard_index(G, X):
    recs = {i:0 for i in G.nodes}
    del recs[X]
    
    for author in recs:
        if not G.has_edge(X, author): # if they are not already friends
            recs[author] = calculateindex(G, X, author)
        else:
            recs[author] = -1 # if they are friends, assign -1 so can remove it
            
    recs = {key:val for key, val in recs.items() if val != -1}
        
    sortedrecs = {k: v for k, v in sorted(recs.items(), key=lambda item: item[1], reverse=True)}
    sortedrecslist = list(sortedrecs.keys())
    return sortedrecslist, sortedrecs

recs, dicts = jaccard_index(G_old, 'Bo Dai')
recs[:15]

['Manabu Kimura',
 'Tingting Zhao',
 'Takafumi Kanamori',
 'Tomoya Sakai',
 'Yao Ma',
 'Nan Du',
 'Taiji Suzuki',
 'Christopher Berlind',
 'Yichen Wang',
 'Ning Xie 0003',
 'Vandana Kanchanapally',
 'Yao Xie 0002',
 'Steven Ehrlich',
 'Emma Cohen',
 'Kaushik Patnaik']

In [79]:
def calculate_adar_index(G, x1, x2):    
    n1 = set(G.neighbors(x1))
    n2 = set(G.neighbors(x2))
    intersection = list(n1.intersection(n2))
    
    for i in range(len(intersection)):
        numneighbors = len(list(G.neighbors(intersection[i])))
        intersection[i] = 1 / (math.log(numneighbors))
    
    index = sum(intersection)
    
    return index

def adamic_adar_index(G, X):
    recs = {i:0 for i in G.nodes}
    del recs[X]
    
    for author in recs:
        if not G.has_edge(X, author): # if they are not already friends
            recs[author] = calculate_adar_index(G, X, author)
        else:
            recs[author] = -1 # if they are friends, assign -1 so can remove it
            
    recs = {key:val for key, val in recs.items() if val != -1}
        
    sortedrecs = {k: v for k, v in sorted(recs.items(), key=lambda item: item[1], reverse=True)}
    sortedrecslist = list(sortedrecs.keys())
    return sortedrecslist, sortedrecs

recs, dicts = adamic_adar_index(G_old, 'Bo Dai')
recs[:15]

['Nan Du',
 'Takafumi Kanamori',
 'Taiji Suzuki',
 'Tomoya Sakai',
 'Yao Ma',
 'Manabu Kimura',
 'Tingting Zhao',
 'Yichen Wang',
 'Shuang Li 0002',
 'Yao Xie 0002',
 'Christopher Berlind',
 'Ichiro Takeuchi',
 'Song Liu 0002',
 'Steven Ehrlich',
 'David P. Woodruff']

In [98]:
def val_10recs(recfunction, oldgraph, newgraph):
    vals = []
    for author in newgraph.nodes:      
        recommendations, dicts = recfunction(oldgraph, author)
        top10_recommendations = recommendations[:10]
        correct = 0
        for x in top10_recommendations:
            if newgraph.has_edge(author, x):
                correct += 1
        vals.append(correct)
    
    avg = sum(vals) / len(vals)
        
    return avg

commonfriends_efficiency = val_10recs(common_friends_number, G_old, G_new)
jaccard_efficiency = val_10recs(jaccard_index, G_old, G_new)
adamic_efficiency = val_10recs(adamic_adar_index, G_old, G_new)

print('Common Friends 10 Recs Efficiency:', commonfriends_efficiency)
print('Jaccards Index 10 Recs Efficiency:', jaccard_efficiency)
print('Adamic/Adar 10 Recs Efficiency:', adamic_efficiency)

Common Friends 10 Recs Efficiency: 0.7804878048780488
Jaccards Index 10 Recs Efficiency: 0.43902439024390244
Adamic/Adar 10 Recs Efficiency: 0.6097560975609756
